In [1]:
import ipywidgets as widgets
from IPython.display import display

widgets.IntSlider()  # test widget

IntSlider(value=0)

In [2]:
# Cell 1: Imports and setup
import pandas as pd
import sqlite3
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

from search_handler import search_laptops
from user_history import log_user_interaction, get_user_history
from agent_handler import query_assistant
from llm_recommendation import generate_recommendation

user_id = "user123"


c:\Users\jagdi\BuyGenie Flask-based e-commerce project\src\agent_handler.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [16]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output, HTML
import sqlite3
import pandas as pd

from search_handler import search_laptops
from llm_recommendation import generate_recommendation

# --- History storage ---
chat_history = []

# --- Widgets ---
query_input = widgets.Textarea(
    placeholder='What kind of laptop are you looking for?',
    description='Query:',
    layout=widgets.Layout(width='100%', height='80px')
)
submit_button = widgets.Button(description="🔍 Search + Recommend", button_style='primary')
output_area = widgets.Output()

# --- Button callback ---
def on_submit(b):
    query = query_input.value.strip()
    if not query:
        with output_area:
            clear_output()
            display(Markdown("❗ Please enter a query."))
        return

    # Process user query
    try:
        # ----- Perform Search -----
        df = search_laptops(query)
        if isinstance(df, str):
            result_md = f"❌ {df}"
        elif df.empty:
            result_md = "⚠️ No laptops matched your query."
        else:
            result_md = "### 🔎 Top 5 Matching Laptops\n"
            for i, (_, row) in enumerate(df.head(5).iterrows()):
                result_md += f"""
**{i+1}. {row['Company']} {row['Product']}**
- 💾 RAM: {row['Ram']}GB | 💽 {row['PrimaryStorage']}GB {row['PrimaryStorageType']}
- 🎮 GPU: {row['GPU_model']} | 💰 Price: €{row['Price_euros']} | ⚖️ {row['Weight']}kg
---
"""

        # ----- Perform Recommendation -----
        conn = sqlite3.connect("db/laptops.db")
        test_df = pd.read_sql_query("SELECT * FROM laptops LIMIT 3", conn)
        conn.close()
        recommendation = generate_recommendation(query, test_df)

        # Save to history
        chat_history.append({
            "query": query,
            "results": result_md,
            "recommendation": recommendation
        })

        # Display all history
        with output_area:
            clear_output()
            display(Markdown("## 🕓 Chat History"))
            for idx, entry in enumerate(chat_history):
                display(Markdown(f"### 🔹 Query {idx + 1}: `{entry['query']}`"))
                display(Markdown(entry["results"]))
                display(HTML(f"<div style='background-color: #4CAF50; color: white; padding: 15px; border-radius: 5px;'>💡 {entry['recommendation']}</div>"))
                display(Markdown("---"))

    except Exception as e:
        with output_area:
            clear_output()
            display(Markdown(f"❗ **Error:** {str(e)}"))

# Hook button
submit_button.on_click(on_submit)

# --- UI Layout ---
display(Markdown("## 💻 BuyGenie - Jupyter Notebook Interface"))
display(query_input, submit_button, output_area)


## 💻 BuyGenie - Jupyter Notebook Interface

Textarea(value='', description='Query:', layout=Layout(height='80px', width='100%'), placeholder='What kind of…

Button(button_style='primary', description='🔍 Search + Recommend', style=ButtonStyle())

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
from agent_handler import query_assistant  # Ensure your function is available

# Create widgets
agent_input = widgets.Textarea(
    placeholder='Ask your question about laptops...',
    description='Your Question:',
    layout=widgets.Layout(width='100%', height='100px')
)

submit_button = widgets.Button(description="💬 Ask Assistant", button_style='success')
agent_chat_output = widgets.Output()
agent_chat_history = []  # Store Q&A history

# Define button click handler
def on_button_click(b):
    question = agent_input.value.strip()
    if not question:
        return

    agent_input.value = ""  # Clear input
    response = query_assistant(question)

    agent_chat_history.append((question, response))

    # Re-render the conversation
    agent_chat_output.clear_output()
    with agent_chat_output:
        display(Markdown("### 🧞 Assistant Chat Log"))
        for q, a in agent_chat_history:
            display(Markdown(f"**You:** {q}"))
            display(Markdown(f"**Assistant:** {a}"))
            display(Markdown("---"))

# Connect the button
submit_button.on_click(on_button_click)

# Display widgets
display(Markdown("### 🧞 Ask the Laptop Assistant"))
display(agent_input, submit_button, agent_chat_output)


### 🧞 Ask the Laptop Assistant

Textarea(value='', description='Your Question:', layout=Layout(height='100px', width='100%'), placeholder='Ask…

Button(button_style='success', description='💬 Ask Assistant', style=ButtonStyle())

Output()